In [ ]:
### https://towardsdatascience.com/the-ultimate-guide-to-a-b-testing-part-4-bayesian-approach-binomial-variables-aa71e25d2165

### to replicate the exact numbers from the article, use the lines prefixed by #EX-code ###
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc3 as pm
import arviz as az

sns.set()

In [ ]:
np.random.seed(87)

N1 = N2 = 2500
K1 = 50
K2 = 42

y1 = np.zeros(N1)
# use choice to ensure no doubles #
y1[np.random.choice(np.arange(len(y1)),replace=False,size=K1)] = 1
#y1[np.random.randint(0,N1,K1)] = 1 #EX-code

y2 = np.zeros(N2)
y2[np.random.choice(np.arange(len(y2)),replace=False,size=K2)] = 1
#y2[np.random.randint(0,N2,K2)] = 1 #EX-code

observations_A = sum(y1)
observations_B = sum(y2)
observations_B

In [ ]:
with pm.Model() as model:

    # define priors
    p_A = pm.Beta('p_A', alpha=K1, beta=N1 - K1)
    p_B = pm.Beta('p_B', alpha=K1, beta=N1 - K1)
    
    # define likelihood
    obs_A = pm.Binomial('obs_A', n=len(y1), p=p_A, observed=observations_A)
    obs_B = pm.Binomial('obs_B', n=len(y2), p=p_B, observed=observations_B)
    
    # define metrics
    pm.Deterministic('difference', p_B - p_A)
    pm.Deterministic('relation', (p_B/p_A) - 1)
    
    trace = pm.sample(progressbar=True)
    #trace = pm.sample(draws=50000, step=pm.Metropolis(), progressbar=True) #EX-code

    #pm.trace_plot(trace[1000:]) #EX-code
    


In [ ]:
with model:
    az.plot_trace(trace)

In [ ]:
with model:
    az.plot_posterior(trace,figsize=(18,6),hdi_prob=0.89,ref_val=0,var_names=['p_A'])
    az.plot_posterior(trace,figsize=(18,6),hdi_prob=0.89,ref_val=0,var_names=['p_B'])
    az.plot_posterior(trace,figsize=(18,6),hdi_prob=0.89,ref_val=0,var_names=['difference'])
    az.plot_posterior(trace,figsize=(18,6),hdi_prob=0.89,ref_val=0,var_names=['relation'])

In [ ]:
with model:
    print (az.summary(trace,hdi_prob=0.89))

In [ ]:
plt.hist(sample_posterior['obs_A'])

In [ ]:
with model:
    az.plot_posterior(trace,ref_val=0,var_names=['difference','relation'])